### Import packeges

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
from matplotlib.ticker import AutoLocator, AutoMinorLocator, LogLocator
import glob
from scipy.interpolate import griddata
from pathlib import Path
import h5py
import sys
from pathlib import Path

# Where am I running?
try:
    # Normal script
    here = Path(__file__).resolve().parent
except NameError:
    # Notebook / REPL
    here = Path.cwd()

phys_const_path = (here / '..' / 'phys_const').resolve()
sys.path.append(str(phys_const_path))

nsm_plots_path = (here / '..' / 'nsm_plots').resolve()
sys.path.append(str(nsm_plots_path))

nsm_plots_postproc = (here / '..' / 'nsm_instabilities').resolve()
sys.path.append(str(nsm_plots_postproc))

import phys_const as pc
import plot_functions as pf
import functions_angular_crossings as fac

### File path

In [ ]:
##################################################################################################################
# Simulation paths

direct = '/home/erick/gw170817_1.00ye_locsim/CFI/cell_32-48-15_dom_1-1-1_km_ncell_1-1-1'
parfile = '/plt00000_particles'
ncell = (1,1,1) # number of cells in each direction
domain = (1e5, 1e5, 1e5) # cm
num_particles_per_energy_bin = 92
num_energy_bins = 13
direct_interpolated_data = direct

direct = '/pscratch/sd/u/uo1999/gw170817_1.00Ye/classical_transport/378_ppEb'
parfile = '/plt00960_particles'
ncell = (96, 96, 64) # number of cells in each direction
domain = (96e5, 96e5, 64e5) # cm
num_particles_per_energy_bin = 378
num_energy_bins = 13
direct_interpolated_data = '/pscratch/sd/u/uo1999/gw170817_1.00Ye/interpolated_SFHo_and_NuLib_quantities'

direct = '/pscratch/sd/u/uo1999/gw170817_1.00Ye/classical_transport/1506_ppEb'
parfile = '/plt00384_particles'
ncell = (96, 96, 64) # number of cells in each direction
domain = (96e5, 96e5, 64e5) # cm
num_particles_per_energy_bin = 1506
num_energy_bins = 13
direct_interpolated_data = '/pscratch/sd/u/uo1999/gw170817_1.00Ye/interpolated_SFHo_and_NuLib_quantities'

##################################################################################################################

all_par_files = sorted(glob.glob(direct + '/plt*_particles'))
plt_numbers = [f.split('/')[-1].split('plt')[1].split('_particles')[0] for f in all_par_files]
plt_numbers = sorted(plt_numbers, key=lambda x: int(x))
all_par_files = [f'plt{plt_num}_particles' for plt_num in plt_numbers]

finaldir = direct.split('/')[-1]

# Calculate the volume of a single cell and the total volume
cellvolume = (domain[0] / ncell[0]) * (domain[1] / ncell[1]) * (domain[2] / ncell[2]) # cm^3
domainvolume = cellvolume * (ncell[0] * ncell[1] * ncell[2]) # cm^3

### Define cell index to be analyzed

In [ ]:
cell_index_i = 15
cell_index_j = 48
cell_index_k = 15

### Cell to be analized

In [ ]:
# Get the cell indices
cell_file_names = glob.glob(direct + parfile + '/cell_*_*_*')
cell_file_names = [file_name.split('/')[-1] for file_name in cell_file_names]
x_cell_ind = np.array([int(file_name.split('_')[1]) for file_name in cell_file_names])
y_cell_ind = np.array([int(file_name.split('_')[2]) for file_name in cell_file_names])
z_cell_ind = np.array([int((file_name.split('_')[3]).split('.')[0]) for file_name in cell_file_names])
cell_indices = np.array(list(zip(x_cell_ind, y_cell_ind, z_cell_ind)))
print('Number of cells:', len(cell_indices))
print(f'shape of cell_indices: {cell_indices.shape}')

# Get the cell indices for ix fix but iy and iz varying all available cells
x_idx_slice = cell_index_i
mask_yz_slice = cell_indices[:,0] == x_idx_slice # fixing the x index in this value
cell_indices_yz_slice = cell_indices[mask_yz_slice]
fig, ax = plt.subplots(figsize=(12, 8))
ax.scatter(cell_indices_yz_slice[:,1], cell_indices_yz_slice[:,2], color='b')
ax.scatter(cell_index_j, cell_index_k, color='r')
ax.set_xlabel('$i_y$')
ax.set_ylabel('$i_z$')
ax.set_title(f'Cell indices for fixed $i_x=${x_idx_slice}')
ax.legend()
# ax.set_xlim([-5,100])
# ax.set_ylim([-5,70])
plt.show()
plt.close(fig)

# Get the cell indices for iy fix but ix and iz varying all available cells
y_idx_slice = cell_index_j
mask_xz_slice = cell_indices[:,1] == y_idx_slice # fixing the y index in this value
cell_indices_xz_slice = cell_indices[mask_xz_slice]
fig, ax = plt.subplots(figsize=(12, 8))
ax.scatter(cell_indices_xz_slice[:,0], cell_indices_xz_slice[:,2], color='b')
ax.scatter(cell_index_i, cell_index_k, color='r')
ax.set_xlabel('$i_x$')
ax.set_ylabel('$i_z$')
ax.set_title(f'Cell indices for fixed $i_y=${y_idx_slice}')
ax.legend()
# ax.set_xlim([-5,100])
# ax.set_ylim([-5,70])
plt.show()
plt.close(fig)

# Get the cell indices for iz fix but ix and iy varying all available cells
z_idx_slice = cell_index_k
mask_xy_slice = cell_indices[:,2] == z_idx_slice # fixing the z index in this value
cell_indices_xy_slice = cell_indices[mask_xy_slice]
fig, ax = plt.subplots(figsize=(12, 8))
ax.scatter(cell_indices_xy_slice[:,0], cell_indices_xy_slice[:,1], color='b')
ax.scatter(cell_index_i, cell_index_j, color='r')
ax.set_xlabel('$i_x$')
ax.set_ylabel('$i_y$')
ax.set_title(f'Cell indices for fixed $i_z=${z_idx_slice}')
ax.legend()
# ax.set_xlim([-5,100])
# ax.set_ylim([-5,70])
plt.show()
plt.close(fig)

# Combine all cell indices from the three slices
# cell_indices_all = np.concatenate((cell_indices_yz_slice, cell_indices_xz_slice, cell_indices_xy_slice), axis=0)
cell_indices_all = np.concatenate((cell_indices_xz_slice, cell_indices_xy_slice), axis=0)
cell_indices_all = np.unique(cell_indices_all, axis=0)

### Getting temperature, electron fraction and density

In [ ]:
rho_ye_T_h5py = h5py.File(direct+'/rho_Ye_T.hdf5', 'r')

Temperature_MeV = np.array(rho_ye_T_h5py['/T_Mev'])[cell_index_i,cell_index_j,cell_index_k]
rho_g_cm3 = np.array(rho_ye_T_h5py['/rho_g|ccm'])[cell_index_i,cell_index_j,cell_index_k]
Ye = np.array(rho_ye_T_h5py['/Ye'])[cell_index_i,cell_index_j,cell_index_k]

print(f'rho_cons = {rho_g_cm3} # g/ccm')
print(f'T_cons = {Temperature_MeV} # MeV')
print(f'Ye_cons = {Ye} # n_electron - n_positron / n_barions')

rho_ye_T_h5py.close()

### Compute ELN number density

In [ ]:
energybinsMeV = np.array([
    1, 3, 5.2382, 8.0097, 11.442, 15.691, 20.953, 27.468,
    35.536, 45.525, 57.895, 73.212, 92.178,
    # 115.66, 144.74, 180.75, 225.33, 280.54
])

energybinstopMeV = np.array([
    2, 4, 6.4765, 9.543, 13.34, 18.042, 23.864, 31.073,
    39.999, 51.052, 64.738, 81.685, 102.67, 
    # 128.65, 160.83, 200.67, 250, 311.08
])

energybinsbottomMeV = np.array([
    0, 2, 4, 6.4765, 9.543, 13.34, 18.042, 23.864, 31.073,
    39.999, 51.052, 64.738, 81.685, 
    # 102.67, 128.65, 160.83, 200.67, 250
])

### Define function to get number density as a function of time

In [ ]:
def get_number_density_per_energy_bins(i, j, k, particlefile):

    parcellname = '/cell_' + str(i) + '_' + str(j) + '_' + str(k) + '.h5'

    particles_h5py = h5py.File(direct+particlefile+parcellname, 'r')
    N00_Re    = np.array(particles_h5py['/N00_Re'   ]) # particles
    N00_Rebar = np.array(particles_h5py['/N00_Rebar']) # particles
    N11_Re    = np.array(particles_h5py['/N11_Re'   ]) # particles
    N11_Rebar = np.array(particles_h5py['/N11_Rebar']) # particles
    N22_Re    = np.array(particles_h5py['/N22_Re'   ]) # particles
    N22_Rebar = np.array(particles_h5py['/N22_Rebar']) # particles
    time_s    = np.array(particles_h5py['/time'     ])[0] # seconds
    pupt      = np.array(particles_h5py['/pupt'     ]) # ergs
    pupx     = np.array(particles_h5py['/pupx'     ]) / pupt # unitless
    pupy     = np.array(particles_h5py['/pupy'     ]) / pupt # unitless
    pupz     = np.array(particles_h5py['/pupz'     ]) / pupt # unitless
    energyMeV = pupt / ( 1e6 * pc.CGSUnitsConst.eV ) # MeV     
    particles_h5py.close()

    n00_Re_split_energy_bins    = np.zeros(len(energybinsbottomMeV)) # particles / cm^3
    n00_Rebar_split_energy_bins = np.zeros(len(energybinsbottomMeV)) # particles / cm^3
    n11_Re_split_energy_bins    = np.zeros(len(energybinsbottomMeV)) # particles / cm^3 
    n11_Rebar_split_energy_bins = np.zeros(len(energybinsbottomMeV)) # particles / cm^3

    for t in range(num_energy_bins):
        energymask = (energyMeV >= energybinsbottomMeV[t]) & (energyMeV < energybinstopMeV[t])
        if np.sum(energymask) != num_particles_per_energy_bin:
            print(f"Warning: Energy bin {t} has {np.sum(energymask)} particles, expected {num_particles_per_energy_bin}")
        n00_Re_split_energy_bins   [t] = np.sum(N00_Re[energymask])    / cellvolume # particles / cm^3
        n00_Rebar_split_energy_bins[t] = np.sum(N00_Rebar[energymask]) / cellvolume # particles / cm^3
        n11_Re_split_energy_bins   [t] = ( np.sum(N11_Re   [energymask]) + np.sum(N22_Re   [energymask]) ) / cellvolume # particles / cm^3
        n11_Rebar_split_energy_bins[t] = ( np.sum(N11_Rebar[energymask]) + np.sum(N22_Rebar[energymask]) ) / cellvolume # particles / cm^3

    return n00_Re_split_energy_bins, n00_Rebar_split_energy_bins, n11_Re_split_energy_bins, n11_Rebar_split_energy_bins, time_s

### Plot energy spectrum to compare with equilibrium distribution

In [ ]:
n00_Re_split_energy_bins, n00_Rebar_split_energy_bins, n11_Re_split_energy_bins, n11_Rebar_split_energy_bins, time_s = get_number_density_per_energy_bins(cell_index_i, cell_index_j, cell_index_k, parfile)

mu_nu_e_MeV_h5py    = h5py.File(direct_interpolated_data+'/mu_nu_e_MeV.h5', 'r')
mu_nubar_e_MeV_h5py = h5py.File(direct_interpolated_data+'/mu_nubar_e_MeV.h5', 'r')
mu_nu_e_MeV    = np.array(mu_nu_e_MeV_h5py   ['/mu_nu_e_MeV']   )[cell_index_i,cell_index_j,cell_index_k]
mu_nubar_e_MeV = np.array(mu_nubar_e_MeV_h5py['/mu_nubar_e_MeV'])[cell_index_i,cell_index_j,cell_index_k]

mu_nu_e_MeV_h5py.close()
mu_nubar_e_MeV_h5py.close()

dE = ( energybinstopMeV - energybinsbottomMeV ) * ( 1e6 * pc.CGSUnitsConst.eV ) # erg
E = ( energybinsMeV ) * ( 1e6 * pc.CGSUnitsConst.eV ) # erg
dOmega = 4.0 * np.pi

feq_nu_e    = 1 / ( 1.0 + np.exp( ( energybinsMeV - mu_nu_e_MeV    ) / Temperature_MeV ) )
feq_nubar_e = 1 / ( 1.0 + np.exp( ( energybinsMeV - mu_nubar_e_MeV ) / Temperature_MeV ) )
feq_nu_x    = 1 / ( 1.0 + np.exp( ( energybinsMeV - 0.0            ) / Temperature_MeV ) )

neq_nu_e    = ( 1 / pc.PhysConst.hc**3 ) * dOmega * dE * E**2 * feq_nu_e
neq_nubar_e = ( 1 / pc.PhysConst.hc**3 ) * dOmega * dE * E**2 * feq_nubar_e
neq_nu_x    = ( 1 / pc.PhysConst.hc**3 ) * dOmega * dE * E**2 * feq_nu_x

f_nu_e    = n00_Re_split_energy_bins    * pc.PhysConst.hc**3 / ( dOmega * dE * E**2 )
f_nubar_e = n00_Rebar_split_energy_bins * pc.PhysConst.hc**3 / ( dOmega * dE * E**2 )
f_nu_x    = n11_Re_split_energy_bins    * pc.PhysConst.hc**3 / ( dOmega * dE * E**2 )

# Print arrays in C++ initializer list format for easy copy-paste
def print_cpp_array(arr, name):
    print(f"double {name}[{len(arr)}] = {{")
    print("    " + ", ".join([f"{x:.16e}" for x in arr]))
    print("};\n")

print_cpp_array(f_nu_e, "f_nu_e")
print_cpp_array(f_nubar_e, "f_nubar_e")
print_cpp_array(f_nu_x, "f_nu_x")
print_cpp_array(energybinsbottomMeV * ( 1e6 * pc.CGSUnitsConst.eV ), "energybinsbottom_ergs")
print_cpp_array(energybinstopMeV * ( 1e6 * pc.CGSUnitsConst.eV ), "energybinstopMeV_ergs")
print_cpp_array(energybinsMeV * ( 1e6 * pc.CGSUnitsConst.eV ), "energybinsMeV_ergs")

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(energybinsMeV, n00_Re_split_energy_bins,    marker='o', label=r'$n_{     \nu_e}$',  color='C0')
ax.plot(energybinsMeV, n00_Rebar_split_energy_bins, marker='o', label=r'$n_{\bar{\nu}_e}$', color='C1')
ax.plot(energybinsMeV, n11_Re_split_energy_bins,    marker='o', label=r'$n_{     \nu_x}$',  color='C2')
ax.plot(energybinsMeV, neq_nu_e,    marker='', label=r'$n_{     \nu_e}^{\mathrm{eq}}$',  color='C0', linestyle='dotted')
ax.plot(energybinsMeV, neq_nubar_e, marker='', label=r'$n_{\bar{\nu}_e}^{\mathrm{eq}}$', color='C1', linestyle='dotted')
ax.plot(energybinsMeV, neq_nu_x,    marker='', label=r'$n_{     \nu_x}^{\mathrm{eq}}$',  color='C2', linestyle='dotted')
ax.set_xlabel(r'$E\,[\mathrm{MeV}]$')
ax.set_ylabel(r'$n\,[\mathrm{cm}^{-3}]$')
leg = ax.legend(framealpha=0.0, ncol=2, fontsize=20)
pf.apply_custom_settings(ax, leg, True)
ax.set_xscale('log')
plt.savefig(f"plots/{finaldir}_n_E.pdf", bbox_inches='tight')
plt.show()
plt.close(fig)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(energybinsMeV, f_nu_e,    marker='o', label=r'$f_{     \nu_e}$')
ax.plot(energybinsMeV, f_nubar_e, marker='o', label=r'$f_{\bar{\nu}_e}$')
ax.plot(energybinsMeV, f_nu_x,    marker='o', label=r'$f_{     \nu_x}$')
ax.set_xlabel(r'$E\,[\mathrm{MeV}]$')
ax.set_ylabel(r'$f\,(E)$')
leg = ax.legend(framealpha=0.0, ncol=1, fontsize=20)
pf.apply_custom_settings(ax, leg, True)
ax.set_xscale('log')
plt.savefig(f"plots/{finaldir}_f_E.pdf", bbox_inches='tight')
plt.show()
plt.close(fig)

plt.figure(figsize=(8, 5))
plt.plot(energybinsMeV, f_nu_e, marker='o', linestyle='-', label=r'$n_{\nu_e}$ (sim)')
plt.plot(energybinsMeV, feq_nu_e, marker='o', linestyle='-', label=r'$n_{\nu_e}$ (eq)')
plt.xlabel('$E$ [MeV]')
plt.ylabel(r'$f_{\nu_e}(E)$')
# plt.xscale('log')
# plt.yscale('log')
plt.legend()
plt.show()

### Plot the time evolution of the neutrino energy spectrum

In [ ]:
all_n00_Re_split_energy_bins    = []
all_n00_Rebar_split_energy_bins = []
all_n11_Re_split_energy_bins    = []
all_n11_Rebar_split_energy_bins = []
tiempo_s = []

for dirpar in all_par_files:
    n00_Re_split_energy_bins, n00_Rebar_split_energy_bins, n11_Re_split_energy_bins, n11_Rebar_split_energy_bins, t_s = get_number_density_per_energy_bins(cell_index_i, cell_index_j, cell_index_k,'/'+dirpar)
    all_n00_Re_split_energy_bins   .append(n00_Re_split_energy_bins)
    all_n00_Rebar_split_energy_bins.append(n00_Rebar_split_energy_bins)
    all_n11_Re_split_energy_bins   .append(n11_Re_split_energy_bins)
    all_n11_Rebar_split_energy_bins.append(n11_Rebar_split_energy_bins)
    tiempo_s.append(t_s)

print("tiempo_s:", tiempo_s)

all_n00_Re_split_energy_bins    = np.array(all_n00_Re_split_energy_bins)
print(f"all_n00_Re_split_energy_bins shape: {all_n00_Re_split_energy_bins.shape}")
all_n00_Rebar_split_energy_bins = np.array(all_n00_Rebar_split_energy_bins)
all_n11_Re_split_energy_bins    = np.array(all_n11_Re_split_energy_bins)
all_n11_Rebar_split_energy_bins = np.array(all_n11_Rebar_split_energy_bins)
tiempo_s = np.array(tiempo_s)

print(f'energybinsMeV = {energybinsMeV}')
print(f'energybinsMeV.shape = {energybinsMeV.shape}')

pf.plot_colored_lines(np.log10(energybinsMeV), np.log10(all_n00_Re_split_energy_bins), tiempo_s, r'$\log(\,E\,[MeV]\,)$', r'$\log(\,n_{\nu_e}\,[\mathrm{cm}^{-3}]\,)$', r'$t \, [s]$', f"plots/{finaldir}_n_ee_E_spectrum.pdf")
pf.plot_colored_lines(np.log10(energybinsMeV), np.log10(all_n00_Rebar_split_energy_bins), tiempo_s, r'$\log(\,E\,[MeV]\,)$', r'$\log(\,n_{\bar{\nu}_e}\,[\mathrm{cm}^{-3}]\,)$', r'$t \, [s]$', f"plots/{finaldir}_n_ebar_E_spectrum.pdf")
pf.plot_colored_lines(np.log10(energybinsMeV), np.log10(all_n11_Re_split_energy_bins), tiempo_s, r'$\log(\,E\,[MeV]\,)$', r'$\log(\,n_{\nu_x}\,[\mathrm{cm}^{-3}]\,)$', r'$t \, [s]$', f"plots/{finaldir}_n_x_E_spectrum.pdf")
pf.plot_colored_lines(np.log10(energybinsMeV), np.log10(all_n11_Rebar_split_energy_bins), tiempo_s, r'$\log(\,E\,[MeV]\,)$', r'$\log(\,n_{\bar{\nu}_x}\,[\mathrm{cm}^{-3}]\,)$', r'$t \, [s]$', f"plots/{finaldir}_n_xbar_E_spectrum.pdf")